# CODE

In [1]:
import sqlalchemy as sqla
from sqlalchemy import create_engine
import traceback
import glob
import os
from pprint import pprint
import simplejson as json
import requests
import time
from IPython.display import display
import pymysql
import sql_metadata
from sqlalchemy import *
import json
import datetime as dt

# CREATE TABLE

In [2]:

URL="dbike.cvo8g1gt1fco.eu-west-1.rds.amazonaws.com"
PORT="3306"
DB = "dbike"
USER = "group15"
PASSWORD = "declanmingbo"
api_key = "53c9b7d9148fef65635074fed863cc14f718219f"
r = requests.get("https://api.jcdecaux.com/vls/v1/stations?contract=dublin&apiKey=" + api_key)

In [3]:
engine = create_engine("mysql+pymysql://{}:{}@{}:{}/{}".format(USER, PASSWORD, URL, PORT, DB), echo=True)
sql = """CREATE DATABASE IF NOT EXISTS dbike;"""
engine.execute(sql)

2022-04-14 00:40:34,438 INFO sqlalchemy.engine.Engine SHOW VARIABLES LIKE 'sql_mode'
2022-04-14 00:40:34,439 INFO sqlalchemy.engine.Engine [raw sql] {}
2022-04-14 00:40:34,450 INFO sqlalchemy.engine.Engine SHOW VARIABLES LIKE 'lower_case_table_names'
2022-04-14 00:40:34,453 INFO sqlalchemy.engine.Engine [generated in 0.00290s] {}
2022-04-14 00:40:34,502 INFO sqlalchemy.engine.Engine SELECT DATABASE()
2022-04-14 00:40:34,503 INFO sqlalchemy.engine.Engine [raw sql] {}
2022-04-14 00:40:34,525 INFO sqlalchemy.engine.Engine CREATE DATABASE IF NOT EXISTS dbike;
2022-04-14 00:40:34,526 INFO sqlalchemy.engine.Engine [raw sql] {}
2022-04-14 00:40:34,535 INFO sqlalchemy.engine.Engine COMMIT


In [4]:
for res in engine.execute("SHOW VARIABLES"):
    print(res)

2022-04-14 00:40:35,231 INFO sqlalchemy.engine.Engine SHOW VARIABLES
2022-04-14 00:40:35,234 INFO sqlalchemy.engine.Engine [raw sql] {}
('activate_all_roles_on_login', 'OFF')
('admin_address', '')
('admin_port', '33062')
('admin_ssl_ca', '')
('admin_ssl_capath', '')
('admin_ssl_cert', '')
('admin_ssl_cipher', '')
('admin_ssl_crl', '')
('admin_ssl_crlpath', '')
('admin_ssl_key', '')
('admin_tls_ciphersuites', '')
('admin_tls_version', 'TLSv1,TLSv1.1,TLSv1.2,TLSv1.3')
('authentication_policy', '*')
('auto_generate_certs', 'ON')
('auto_increment_increment', '1')
('auto_increment_offset', '1')
('autocommit', 'OFF')
('automatic_sp_privileges', 'ON')
('avoid_temporal_upgrade', 'OFF')
('back_log', '65')
('basedir', '/rdsdbbin/mysql-8.0.27.R3/')
('big_tables', 'OFF')
('bind_address', '*')
('binlog_cache_size', '32768')
('binlog_checksum', 'CRC32')
('binlog_direct_non_transactional_updates', 'OFF')
('binlog_encryption', 'OFF')
('binlog_error_action', 'ABORT_SERVER')
('binlog_expire_logs_seconds

In [5]:
def initaldatabase():
    sql = """
    CREATE TABLE IF NOT EXISTS station (
    address VARCHAR(256) ,
    banking INTEGER,
    bike_stands INTEGER,
    bonus INTEGER,
    contract_name VARCHAR(256),
    name VARCHAR(256),
    number INTEGER,
    position_lat REAL,
    position_lng REAL,
    status VARCHAR(256))"""
    
    try:
        res = engine.execute ("DROP TABLE IF EXISTS station")
        res = engine.execute(sql)
        print(res)
    except Exception as e:
        print(e)
        
    sql= """
    CREATE TABLE IF NOT EXISTS availability (
    number INTEGER,
    available_bikes INTEGER,
    available_bike_stands INTEGER,
    last_update VARCHAR(256))"""
    try:
        res = engine.execute ("DROP TABLE IF EXISTS availability")
        res = engine.execute(sql)
        print(res)
    except Exception as e:
        print(e)
    
    station_insert()


# INSERT DATA

In [6]:
def stations_to_db():
    stationList=[]
    stations = json.loads(r.text)
    for station in stations:
        vals_s = (station.get('address'), 
                int(station.get('banking')),
                station.get('bike_stands'), 
                int(station.get('bonus')),
                station.get('contract_name'),
                station.get('name'),
                station.get('number'),
                station.get('position').get('lat'),
                station.get('position').get('lng'),
                station.get('status'))
        stationList.append(vals_s)
    return stationList



In [9]:
def station_insert():
    vals = stations_to_db()
    try:
        for val in vals:
            engine.execute("INSERT INTO station values(%s,%s,%s,%s,%s,%s,%s,%s,%s,%s)", val)
            db.commit()
            print("Insert correctly!")
    except Exception as e:
        db.rollback()
        print(e)

In [10]:
def availability_to_db():
    availabilityList=[]
    stations = json.loads(r.text)
    for availability in stations:
        vals_a = (availability.get('number'),
               availability.get('available_bikes'),
               availability.get('available_bike_stands'),
               dt.datetime.fromtimestamp(int(availability.get('last_update') / 1e3)))
        
        availabilityList.append(vals_a)
    return availabilityList

In [11]:
def availability_insert():
    vala = availability_to_db()
    try:
        for val in vala:
            sql = """INSERT INTO dbike.availability (number,available_bikes,available_bike_stands,last_update) VALUE (%s,%s,%s,'%s')""" % val
            engine.execute(sql)
            print("Insert correctly!")
    except Exception as e:
        print(e)
        
    db.close()

In [ ]:
import requests 
import traceback 
import datetime
import time
import mysql
from datetime import datetime

import mysql.connector
import sys

URL="dbike.cvo8g1gt1fco.eu-west-1.rds.amazonaws.com"
PORT=3306
DB = "dbike"
USER = "group15"
PASSWORD = "declanmingbo"

db = pymysql.connect(
    host=URL,
    user=USER,
    password=PASSWORD,
    port=PORT,
    database=DB)
cursor = db.cursor()
initaldatabase()
db.close()
while True:
    
    api_key = "53c9b7d9148fef65635074fed863cc14f718219f"
    r = requests.get("https://api.jcdecaux.com/vls/v1/stations?contract=dublin&apiKey=" + api_key)
    
    db = pymysql.connect(
    host=URL,
    user=USER,
    password=PASSWORD,
    port=PORT,
    database=DB)
    cursor = db.cursor()
    
    availability_insert()
    
    time.sleep(5*60)
    
    



2022-04-14 00:40:52,733 INFO sqlalchemy.engine.Engine DROP TABLE IF EXISTS station
2022-04-14 00:40:52,734 INFO sqlalchemy.engine.Engine [raw sql] {}
2022-04-14 00:40:52,753 INFO sqlalchemy.engine.Engine COMMIT
2022-04-14 00:40:52,762 INFO sqlalchemy.engine.Engine 
    CREATE TABLE IF NOT EXISTS station (
    address VARCHAR(256) ,
    banking INTEGER,
    bike_stands INTEGER,
    bonus INTEGER,
    contract_name VARCHAR(256),
    name VARCHAR(256),
    number INTEGER,
    position_lat REAL,
    position_lng REAL,
    status VARCHAR(256))
2022-04-14 00:40:52,764 INFO sqlalchemy.engine.Engine [raw sql] {}
2022-04-14 00:40:52,796 INFO sqlalchemy.engine.Engine COMMIT
2022-04-14 00:40:52,810 INFO sqlalchemy.engine.Engine DROP TABLE IF EXISTS availability
2022-04-14 00:40:52,813 INFO sqlalchemy.engine.Engine [raw sql] {}
2022-04-14 00:40:52,833 INFO sqlalchemy.engine.Engine COMMIT
2022-04-14 00:40:52,846 INFO sqlalchemy.engine.Engine 
    CREATE TABLE IF NOT EXISTS availability (
    number